In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')
# Print system information to confirm it's working
ee.ImageCollection("MODIS/006/MCD43A3").first().getInfo()


# Define the GPS location
# point = ee.Geometry.Point([longitude, latitude])
point = ee.Geometry.Point([-1.48067, 15.34322])


# Load MODIS Albedo dataset (8-day product)
modis_albedo = ee.ImageCollection("MODIS/006/MCD43A3") \
    .filterBounds(point) \
    .sort("system:time_start", False)  # Get the latest available image

# Select the first available image - you can use BSA (direct radiation) or WSA (diffuse radiation) depending on wether you want the white sky or black sky
latest_albedo = modis_albedo.first().select("Albedo_WSA_Band1")

# Extract albedo value at location
albedo_value = latest_albedo.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=point,
    scale=500,  # 500m resolution
    bestEffort=True
)

print("MODIS Albedo:", albedo_value.getInfo())

# divide value by 1000 to get the actual because MODIS albedo values are scaled by a factor of 1000.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define multiple sites (Longitude, Latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_MON": [23.2525, -15.4391],
    "ZA_WGN": [26.9391, -26.56944]

    # Add the rest of your 16 sites here
}

# Load MODIS Albedo dataset
modis_albedo = ee.ImageCollection("MODIS/006/MCD43A3") \
    .sort("system:time_start", False)  # Get the latest available image

# Extract albedo for each site
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)

    # Select the first available image - White-Sky Albedo (WSA) or Black-Sky Albedo (BSA)
    latest_albedo = modis_albedo.first().select("Albedo_WSA_Band1")

    # Extract albedo value at location
    albedo_value = latest_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,  # 500m resolution
        bestEffort=True
    ).getInfo()

    # Store results
    results[site] = albedo_value["Albedo_WSA_Band1"] / 1000 if albedo_value["Albedo_WSA_Band1"] else None

# Print results
# Print results with error handling for None values
for site, albedo in results.items():
    if albedo is not None:
        print(f"{site} Albedo: {albedo:.3f}")
    else:
        print(f"{site} Albedo: No data available")


In [ ]:
# Load MODIS Albedo dataset and compute long-term mean
modis_albedo = ee.ImageCollection("MODIS/006/MCD43A3") \
    .filterBounds(point) \
    .filterDate("2005-01-01", "2023-12-31") \
    .select("Albedo_WSA_Band1") \
    .mean()  # Compute long-term mean

# Extract albedo value at location
albedo_value = modis_albedo.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=point,
    scale=500,
    bestEffort=True
).getInfo()

# Extract and scale value
if albedo_value and "Albedo_WSA_Band1" in albedo_value:
    albedo = albedo_value["Albedo_WSA_Band1"] / 1000
    print(f"Long-Term MODIS Albedo (2005-2023) at (-2.6942, 5.2685): {albedo:.3f}")
else:
    print("No albedo data available for this location.")


In [ ]:
import ee

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

# Define multiple sites (Longitude, Latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    #"GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_MON": [23.2525, -15.4391],
    "ZA_WGN": [26.9391, -26.56944]
}

# Load MODIS Albedo dataset and get the latest image
modis_albedo = ee.ImageCollection("MODIS/006/MCD43A3") \
    .sort("system:time_start", False)  # Get the most recent available image

# Select the latest available MODIS albedo image
latest_albedo = modis_albedo.first().select(["Albedo_BSA_Band1", "Albedo_WSA_Band1"])

# Extract albedo for each site
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)

    # Extract albedo values at location
    albedo_values = latest_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,
        bestEffort=True
    ).getInfo()

    # Check if data exists
    if "Albedo_BSA_Band1" in albedo_values and "Albedo_WSA_Band1" in albedo_values:
        BSA = albedo_values["Albedo_BSA_Band1"] / 1000  # Convert from MODIS scale
        WSA = albedo_values["Albedo_WSA_Band1"] / 1000
        Mean_Albedo = (BSA + WSA) / 2  # Compute mean
        results[site] = {"BSA": BSA, "WSA": WSA, "Mean": Mean_Albedo}
    else:
        results[site] = {"BSA": None, "WSA": None, "Mean": None}

# Print results
for site, albedo_data in results.items():
    BSA = albedo_data["BSA"]
    WSA = albedo_data["WSA"]
    Mean_Albedo = albedo_data["Mean"]
    
    if BSA is not None and WSA is not None:
        print(f"{site} - BSA: {BSA:.3f}, WSA: {WSA:.3f}, Mean Albedo: {Mean_Albedo:.3f}")
    else:
        print(f"{site} - No data available")


In [ ]:
import ee

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define multiple sites (Longitude, Latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "UG_JIN": [33.18333333, 0.4],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_MON": [23.2525, -15.4391],
    "ZA_WGN": [26.9391, -26.56944]
}

# Load MODIS Albedo dataset (mean over 2000-2025)
modis_albedo = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterDate("2000-01-01", "2025-02-28") \
    .select(["Albedo_BSA_Band1", "Albedo_WSA_Band1"]) \
    .mean()  # Compute long-term mean

# Extract albedo for each site
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)

    # Extract BSA and WSA values
    albedo_values = modis_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,
        bestEffort=True
    ).getInfo()

    # Check if data exists
    if "Albedo_BSA_Band1" in albedo_values and "Albedo_WSA_Band1" in albedo_values:
        BSA = albedo_values["Albedo_BSA_Band1"] / 1000  # Convert from MODIS scale
        WSA = albedo_values["Albedo_WSA_Band1"] / 1000
        Mean_Albedo = (BSA + WSA) / 2  # Compute mean
        results[site] = {"BSA": BSA, "WSA": WSA, "Mean": Mean_Albedo}
    else:
        results[site] = {"BSA": None, "WSA": None, "Mean": None}

# Print results
for site, albedo_data in results.items():
    BSA = albedo_data["BSA"]
    WSA = albedo_data["WSA"]
    Mean_Albedo = albedo_data["Mean"]
    
    if BSA is not None and WSA is not None:
        print(f"{site} - BSA: {BSA:.3f}, WSA: {WSA:.3f}, Mean Albedo: {Mean_Albedo:.3f}")
    else:
        print(f"{site} - No data available")


In [ ]:
import ee

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define sites (Longitude, Latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "UG_JIN": [33.18333333, 0.4],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_WGN": [26.9391, -26.56944],
    "ZM_MON": [23.2525, -15.4391]
}

# Get mean NDVI for 2000–2023
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1") \
    .filterDate("2022-12-01", "2023-02-28") \
    .select("NDVI") \
    .mean()

# Mask out vegetated areas (NDVI > 0.3 → 3000 in MODIS scale)
bare_mask = ndvi.lt(3000)

# Load MODIS albedo and apply bare soil mask
modis_albedo = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterDate("2000-01-01", "2023-12-31") \
    .select(["Albedo_BSA_Band1", "Albedo_WSA_Band1"]) \
    .mean() \
    .updateMask(bare_mask)  # Apply bare soil mask

# Extract albedo for each site
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)

    # Reduce region with mask applied
    albedo_values = modis_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,
        bestEffort=True
    ).getInfo()

    # Get albedo values (check for None)
    BSA = albedo_values.get("Albedo_BSA_Band1")
    WSA = albedo_values.get("Albedo_WSA_Band1")

    if BSA is not None and WSA is not None:
        BSA = BSA / 1000
        WSA = WSA / 1000
        Mean_Albedo = (BSA + WSA) / 2
        results[site] = {"BSA": BSA, "WSA": WSA, "Mean": Mean_Albedo}
    else:
        results[site] = {"BSA": None, "WSA": None, "Mean": None}

# Print results
for site, albedo_data in results.items():
    BSA = albedo_data["BSA"]
    WSA = albedo_data["WSA"]
    Mean_Albedo = albedo_data["Mean"]

    if BSA is not None or WSA is not None:
        if BSA is not None:
            print(f"{site} - BSA (bare soil): {BSA:.3f}", end=", ")
        else:
            print(f"{site} - BSA (bare soil): None", end=", ")

        if WSA is not None:
            print(f"WSA: {WSA:.3f}", end=", ")
        else:
            print(f"WSA: None", end=", ")

        if BSA is not None and WSA is not None:
            print(f"Mean Albedo: {Mean_Albedo:.3f}")
        else:
            print("Mean Albedo: N/A (incomplete)")
    else:
        print(f"{site} - No bare soil albedo available")


In [ ]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define flux tower sites with coordinates (Longitude, Latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "UG_JIN": [33.18333333, 0.4],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_WGN": [26.9391, -26.56944],
    "ZM_MON": [23.2525, -15.4391]
}

# Load MODIS LAI data (MOD15A2H), filter to dry season (example: Dec–Feb)
lai = ee.ImageCollection("MODIS/061/MOD15A2H") \
    .filterDate("2022-12-01", "2023-02-28") \
    .select("Lai_500m") \
    .mean()

# Mask out vegetated areas (keep only pixels where LAI < 0.5)
bare_mask = lai.lt(0.5)

# Load MODIS albedo (long-term mean) and apply LAI-based mask
modis_albedo = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterDate("2000-01-01", "2023-12-31") \
    .select(["Albedo_BSA_Band1", "Albedo_WSA_Band1"]) \
    .mean() \
    .updateMask(bare_mask)

# Extract bare soil albedo at each site
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)

    # Reduce albedo image at the point location
    albedo_values = modis_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,
        bestEffort=True
    ).getInfo()

    # Get values and scale from MODIS units
    BSA = albedo_values.get("Albedo_BSA_Band1")
    WSA = albedo_values.get("Albedo_WSA_Band1")

    # Store results with scaled albedo values
    if BSA is not None:
        BSA = BSA / 1000
    if WSA is not None:
        WSA = WSA / 1000
    if BSA is not None and WSA is not None:
        Mean = (BSA + WSA) / 2
    else:
        Mean = None

    results[site] = {"BSA": BSA, "WSA": WSA, "Mean": Mean}

# Print results
for site, albedo_data in results.items():
    BSA = albedo_data["BSA"]
    WSA = albedo_data["WSA"]
    Mean = albedo_data["Mean"]

    if BSA is not None or WSA is not None:
        if BSA is not None:
            print(f"{site} - BSA (bare soil): {BSA:.3f}", end=", ")
        else:
            print(f"{site} - BSA (bare soil): None", end=", ")

        if WSA is not None:
            print(f"WSA: {WSA:.3f}", end=", ")
        else:
            print(f"WSA: None", end=", ")

        if Mean is not None:
            print(f"Mean Albedo: {Mean:.3f}")
        else:
            print("Mean Albedo: N/A (incomplete)")
    else:
        print(f"{site} - No bare soil albedo available")


In [ ]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define flux tower sites with coordinates (Longitude, Latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "UG_JIN": [33.18333333, 0.4],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_WGN": [26.9391, -26.56944],
    "ZM_MON": [23.2525, -15.4391]
}

# Site-specific dry season windows for LAI masking
site_lai_windows = {
    "GH_ANK": ("2022-12-01", "2023-02-28"),
    "ML_AGG": ("2022-12-01", "2023-02-28"),
    "NE_WAF": ("2022-12-01", "2023-02-28"),
    "NE_WAM": ("2022-12-01", "2023-02-28"),
    "SN_DHR": ("2022-12-01", "2023-02-28"),
    "SN_NKR": ("2022-12-01", "2023-02-28"),
    "SN_RAG": ("2022-12-01", "2023-02-28"),
    "SD_DEM": ("2022-11-01", "2023-03-31"),
    "CG_TCH": ("2022-12-01", "2023-02-28"),
    "UG_JIN": ("2022-12-01", "2023-02-28"),
    "BW_GUM": ("2022-05-01", "2022-10-31"),
    "BW_NXR": ("2022-05-01", "2022-10-31"),
    "ZA_CATH": ("2022-05-01", "2022-10-31"),
    "ZA_KRU": ("2022-05-01", "2022-10-31"),
    "ZA_WGN": ("2022-05-01", "2022-10-31"),
    "ZM_MON": ("2022-05-01", "2022-10-31")
}

# Load MODIS albedo (long-term mean)
modis_albedo = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterDate("2000-01-01", "2023-12-31") \
    .select(["Albedo_BSA_Band1", "Albedo_WSA_Band1"]) \
    .mean()

# Extract bare soil albedo using relaxed LAI threshold
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)
    start, end = site_lai_windows[site]

    # Load MODIS LAI for the site-specific dry season
    lai = ee.ImageCollection("MODIS/061/MOD15A2H") \
        .filterDate(start, end) \
        .select("Lai_500m") \
        .mean()

    # Mask pixels where LAI >= 1.0
    bare_mask = lai.lt(1.0)

    # Apply mask to albedo image
    masked_albedo = modis_albedo.updateMask(bare_mask)

    # Reduce region to get pixel value at site
    albedo_values = masked_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,
        bestEffort=True
    ).getInfo()

    # Extract and scale albedo values
    BSA = albedo_values.get("Albedo_BSA_Band1")
    WSA = albedo_values.get("Albedo_WSA_Band1")

    if BSA is not None:
        BSA = BSA / 1000
    if WSA is not None:
        WSA = WSA / 1000
    if BSA is not None and WSA is not None:
        Mean = (BSA + WSA) / 2
    else:
        Mean = None

    results[site] = {"BSA": BSA, "WSA": WSA, "Mean": Mean}

# Print results
for site, albedo_data in results.items():
    BSA = albedo_data["BSA"]
    WSA = albedo_data["WSA"]
    Mean = albedo_data["Mean"]

    if BSA is not None or WSA is not None:
        if BSA is not None:
            print(f"{site} - BSA (bare soil): {BSA:.3f}", end=", ")
        else:
            print(f"{site} - BSA (bare soil): None", end=", ")

        if WSA is not None:
            print(f"WSA: {WSA:.3f}", end=", ")
        else:
            print(f"WSA: None", end=", ")

        if Mean is not None:
            print(f"Mean Albedo: {Mean:.3f}")
        else:
            print("Mean Albedo: N/A (incomplete)")
    else:
        print(f"{site} - No bare soil albedo available")


In [ ]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define sites and coordinates
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "UG_JIN": [33.18333333, 0.4],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_WGN": [26.9391, -26.56944],
    "ZM_MON": [23.2525, -15.4391]
}

# Site-specific NDVI dry season windows
site_ndvi_windows = {
    "GH_ANK": ("2022-12-01", "2023-02-28"),
    "ML_AGG": ("2022-12-01", "2023-02-28"),
    "NE_WAF": ("2022-12-01", "2023-02-28"),
    "NE_WAM": ("2022-12-01", "2023-02-28"),
    "SN_DHR": ("2022-12-01", "2023-02-28"),
    "SN_NKR": ("2022-12-01", "2023-02-28"),
    "SN_RAG": ("2022-12-01", "2023-02-28"),
    "SD_DEM": ("2022-11-01", "2023-03-31"),
    "CG_TCH": ("2022-12-01", "2023-02-28"),
    "UG_JIN": ("2022-12-01", "2023-02-28"),
    "BW_GUM": ("2022-05-01", "2022-10-31"),
    "BW_NXR": ("2022-05-01", "2022-10-31"),
    "ZA_CATH": ("2022-05-01", "2022-10-31"),
    "ZA_KRU": ("2022-05-01", "2022-10-31"),
    "ZA_WGN": ("2022-05-01", "2022-10-31"),
    "ZM_MON": ("2022-05-01", "2022-10-31")
}

# Load MODIS albedo (long-term mean)
modis_albedo = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterDate("2000-01-01", "2023-12-31") \
    .select(["Albedo_BSA_Band1", "Albedo_WSA_Band1"]) \
    .mean()

# Extract bare soil albedo using NDVI masking
results = {}
for site, coords in sites.items():
    point = ee.Geometry.Point(coords)
    start, end = site_ndvi_windows[site]

    # Load MODIS NDVI for the dry season
    ndvi = ee.ImageCollection("MODIS/061/MOD13Q1") \
        .filterDate(start, end) \
        .select("NDVI") \
        .mean()

    # Mask areas with NDVI >= 0.3 (MODIS NDVI scale: 0.3 * 10000 = 3000)
    bare_mask = ndvi.lt(6000)

    # Apply NDVI mask to albedo
    masked_albedo = modis_albedo.updateMask(bare_mask)

    # Reduce region to extract albedo at point
    albedo_values = masked_albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=500,
        bestEffort=True
    ).getInfo()

    # Scale and store albedo values
    BSA = albedo_values.get("Albedo_BSA_Band1")
    WSA = albedo_values.get("Albedo_WSA_Band1")

    if BSA is not None:
        BSA = BSA / 1000
    if WSA is not None:
        WSA = WSA / 1000
    if BSA is not None and WSA is not None:
        Mean = (BSA + WSA) / 2
    else:
        Mean = None

    results[site] = {"BSA": BSA, "WSA": WSA, "Mean": Mean}

# Print results
for site, albedo_data in results.items():
    BSA = albedo_data["BSA"]
    WSA = albedo_data["WSA"]
    Mean = albedo_data["Mean"]

    if BSA is not None or WSA is not None:
        if BSA is not None:
            print(f"{site} - BSA (bare soil): {BSA:.3f}", end=", ")
        else:
            print(f"{site} - BSA (bare soil): None", end=", ")

        if WSA is not None:
            print(f"WSA: {WSA:.3f}", end=", ")
        else:
            print(f"WSA: None", end=", ")

        if Mean is not None:
            print(f"Mean Albedo: {Mean:.3f}")
        else:
            print("Mean Albedo: N/A (incomplete)")
    else:
        print(f"{site} - No bare soil albedo available")


In [ ]:
import ee

# Authenticate & initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Define sites: (longitude, latitude)
sites = {
    "BW_GUM": [22.37111111, -18.96472222],
    "BW_NXR": [23.17916667, -19.54805556],
    "CG_TCH": [11.65641667, -4.289166667],
    "GH_ANK": [-2.69420592, 5.26854275],
    "ML_AGG": [-1.48067, 15.34322],
    "NE_WAF": [2.63368891, 13.64758138],
    "NE_WAM": [2.62984816, 13.644017],
    "SD_DEM": [30.4783, 13.2829],
    "SN_DHR": [-15.43222, 15.40278],
    "SN_NKR": [-16.45357, 14.49581],
    "SN_RAG": [-16.4563, 14.4944],
    "UG_JIN": [33.18333333, 0.4],
    "ZA_CATH": [29.2359, -28.9755],
    "ZA_KRU": [31.4969, -25.0197],
    "ZA_WGN": [26.9391, -26.56944],
    "ZM_MON": [23.2525, -15.4391]
}

# Define dry season window (adjust if needed)
start_date = '2022-06-01'
end_date = '2022-08-31'

# Storage for results
results = {}

for site, coords in sites.items():
    print(f"Processing {site}...")

    # Define geometry: 60 m buffer around flux tower
    point = ee.Geometry.Point(coords)
    buffer = point.buffer(60)

    # Load Sentinel-2 SR collection
    s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(point) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

    # Skip site if no images are found
    if s2_col.size().getInfo() == 0:
        results[site] = None
        print(f"{site}: No Sentinel-2 data available in this period.")
        continue

    # Merge into a single image
    s2 = s2_col.mosaic()

    # NDVI mask
    ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    bare_mask = ndvi.lt(0.3)
    masked = s2.updateMask(bare_mask)

    # Scale reflectance to 0–1
    scaled = masked.select(['B2', 'B3', 'B4', 'B8']).divide(10000)

    # Liang (2001) broadband shortwave albedo formula
    albedo = scaled.expression(
        '0.356*B2 + 0.130*B3 + 0.373*B4 + 0.085*B8 - 0.0018',
        {
            'B2': scaled.select('B2'),
            'B3': scaled.select('B3'),
            'B4': scaled.select('B4'),
            'B8': scaled.select('B8')
        }
    ).rename('albedo')

    # Reduce region (buffered site)
    albedo_value = albedo.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=20,
        bestEffort=True
    ).getInfo()

    # Store result
    results[site] = albedo_value.get('albedo')

# Print final results
print("\n📊 Sentinel-2 Derived Bare Soil Albedo (NDVI < 0.3, 60m buffer):")
for site, val in results.items():
    if val is not None:
        print(f"{site}: {val:.3f}")
    else:
        print(f"{site}: No data (masked or unavailable)")


In [1]:
import ee

# Authenticate & initialize
ee.Authenticate()
ee.Initialize(project='ee-akhasfaith')

# Site coordinates (ZA_KRU)
point = ee.Geometry.Point([31.4969, -25.0197])
buffer = point.buffer(60)

# Date range (one full year or more for seasonal signal)
start_date = '2022-01-01'
end_date = '2022-12-31'

# Load MODIS LAI (MOD15A2H) and scale (scale factor = 0.1)
lai_col = ee.ImageCollection("MODIS/061/MOD15A2H") \
    .filterDate(start_date, end_date) \
    .select("Lai_500m") \
    .map(lambda img: img.multiply(0.1).copyProperties(img, ['system:time_start']))

# Load MODIS white-sky albedo (MCD43A3, Band 1) and scale (scale = 0.001)
albedo_col = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterDate(start_date, end_date) \
    .select("Albedo_WSA_Band1") \
    .map(lambda img: img.multiply(0.001).copyProperties(img, ['system:time_start']))

# Join LAI and Albedo by time
def join_lai_albedo(lai_img):
    date = lai_img.date()
    match = albedo_col.filterDate(date.advance(-1, 'day'), date.advance(1, 'day')).first()
    return lai_img.addBands(match)

# Create time series with both LAI and albedo
time_series = lai_col.map(join_lai_albedo)

# Filter out incomplete observations
time_series = time_series.map(lambda img: img.updateMask(
    img.select("Albedo_WSA_Band1").mask().And(img.select("Lai_500m").mask()))
)

# Extract LAI + albedo values over buffer as a FeatureCollection
features = time_series.map(lambda img: ee.Feature(buffer.centroid(), {
    'LAI': img.select('Lai_500m').reduceRegion(
        reducer=ee.Reducer.mean(), geometry=buffer, scale=500).get('Lai_500m'),
    'Albedo': img.select('Albedo_WSA_Band1').reduceRegion(
        reducer=ee.Reducer.mean(), geometry=buffer, scale=500).get('Albedo_WSA_Band1'),
    'date': img.date().format('YYYY-MM-dd')
}))

# Export-ready FeatureCollection
print(features.limit(5).getInfo())  # Show a few rows

# (Optional) Export to CSV via GEE Tasks or manually save if running in JS code editor


{'type': 'FeatureCollection', 'columns': {}, 'version': 1744356061849381, 'id': 'MODIS/061/MOD15A2H', 'properties': {'system:is_global': 1}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.496900002112806, -25.019699938235185]}, 'id': '2022_01_01', 'properties': {'Albedo': 0.049, 'LAI': 2, 'date': '2022-01-01'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.496900002112806, -25.019699938235185]}, 'id': '2022_01_09', 'properties': {'Albedo': 0.047, 'LAI': 2.1, 'date': '2022-01-09'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.496900002112806, -25.019699938235185]}, 'id': '2022_01_17', 'properties': {'Albedo': 0.049, 'LAI': 0.9, 'date': '2022-01-17'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.496900002112806, -25.019699938235185]}, 'id': '2022_01_25', 'properties': {'Albedo': None, 'LAI': None, 'date': '2022-01-25'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinat